# Scrape stat data

References:
* Main source of scraped statistics: http://japan-mk.blog.jp/
* Additional source of statistics: https://www.mariowiki.com/Mario_Kart_8_Deluxe_in-game_statistics

## Initial setup

### Import libraries

In [1]:
# Async-related imports
import asyncio
import nest_asyncio

# Web scraping imports
import urllib.request
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

# Data processing imports
import os
from pathlib import Path
import json
import numpy as np
import pandas as pd

### Pre-initialization

In [2]:
# Utility functions
def read_json(filename):
    with open(filename) as fp:
        return json.load(fp)
    
# Pre-initialization
IS_RUNNING_NOTEBOOK = '__file__' not in globals()
IS_RUNNING_BOKEH = __name__.startswith('bokeh')

MAIN_DIRECTORY = os.path.abspath('') if IS_RUNNING_NOTEBOOK else os.path.dirname(__file__)
ROOT_DIRECTORY = str(Path(MAIN_DIRECTORY).parent.absolute())

nest_asyncio.apply()

# Load Japanese-to-English translations
JA_EN_TRANSLATE_TECHNICAL  = read_json(ROOT_DIRECTORY + '/translations/ja_en/technical.json')
JA_EN_TRANSLATE_CHARACTERS = read_json(ROOT_DIRECTORY + '/translations/ja_en/characters.json')
JA_EN_TRANSLATE_FRAMES     = read_json(ROOT_DIRECTORY + '/translations/ja_en/frames.json')
JA_EN_TRANSLATE_TIRES      = read_json(ROOT_DIRECTORY + '/translations/ja_en/tires.json')
JA_EN_TRANSLATE_GLIDERS    = read_json(ROOT_DIRECTORY + '/translations/ja_en/gliders.json')

## Run main logic

### Define low-level logic

In [3]:
async def fetch_page_sources(urls, stat_tab=3):
    page_sources = []
    
    async with async_playwright() as p:
        # Open a new browser
        browser = await p.chromium.launch()
        
        # Open each URL in a new tab to conserve resources
        for (i, url) in enumerate(urls):
            page = await browser.new_page()
            await page.goto(url)

            # Change stat tab
            href = await page.evaluate(f'() => $(".sw_sec{stat_tab}").click()')

            # Grab HTML content
            page_sources.append(await page.content())
            
        # Close the browser session once we're done
        await browser.close()
    
    return page_sources


def extract_table_data_from_src(html_doc, translation_table):
    html_soup = BeautifulSoup(html_doc, 'html.parser')
    main_table = html_soup.select_one('table')

    # Image tags don't have a proper text string to help identify each row, so we're extracting it from the 'alt' attribute
    for img_tag in main_table.select('tbody > tr > td > img'):
        img_tag.replace_with(img_tag['alt'], ' / ')

    # Replace Japanese text with English
    translated_main_table = str(main_table)
    for (ja_text, en_text) in translation_table.items():
        translated_main_table = translated_main_table.replace(ja_text, en_text)

    # Use Pandas to parse the table and standardize the dataframe
    table_df = pd.read_html(translated_main_table, flavor='html5lib')[0]
    table_df = standardize_dataframe(table_df)
    return table_df


def standardize_dataframe(df):
    def _to_numeric(cell):
        try:
            return float(cell)
        except ValueError:
            return cell
    
    def _remove_dupes(arr):
        return list(dict.fromkeys(arr))
    
    # Convert all cells to numeric if possible
    df = df.applymap(_to_numeric)
    
    # Merge the multi-index to a single index
    df = df.set_axis(
        np.apply_along_axis(
            # HACK: We need to wrap the row in an numpy array since a string is an
            # array of characters and the output of the function expects a 1D array of sorts
            # Additionally, not casting it as an 'object' will trim the string contents to the shortest string size
            lambda row: np.array(" - ".join( _remove_dupes(row) ), dtype='object'),
            axis=1,
            arr=np.array(df.columns.tolist(), dtype = 'object')
        ),
        axis='columns'
    )
    
    # Remove rows that have all strings. This works by collecting all present types across each row, getting its
    # string representation, removing the 'str' type and checking if any other types remain
    df = df[
        np.apply_along_axis(
            func1d=lambda row: len(set([type(cell).__name__ for cell in row]) - {'str'}),
            axis=1,
            arr=df.to_numpy()
        ) == 1
    ]
    
    # Remove the ending slashes from the first column
    df.iloc[:, 0] = df.iloc[:, 0].str.rstrip(to_strip=' /')
    
    return df

### Scrape stats from japan-mk.blog.jp

In [4]:
async def scrape_sources():
    return await fetch_page_sources([
        'http://japan-mk.blog.jp/mk8dx.st-c', # Characters
        'http://japan-mk.blog.jp/mk8dx.st-f', # Frames
        'http://japan-mk.blog.jp/mk8dx.st-t', # Tires
        'http://japan-mk.blog.jp/mk8dx.st-g', # Gliders
    ], stat_tab=1)

MAIN_LOOP = asyncio.get_event_loop()

print('Fetching all page statistics sources...', end='')

(
    char_stat_src,
    frame_stat_src,
    tire_stat_src,
    glider_stat_src,
) = MAIN_LOOP.run_until_complete(scrape_sources())

print('done!')

Fetching all page statistics sources...done!


### Extract data from sources

In [5]:
print('Extracting tabular data from sources...', end='')

character_stats_df = extract_table_data_from_src(char_stat_src,   JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_CHARACTERS)
frame_stats_df     = extract_table_data_from_src(frame_stat_src,  JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_FRAMES)
tire_stats_df      = extract_table_data_from_src(tire_stat_src,   JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_TIRES)
glider_stats_df    = extract_table_data_from_src(glider_stat_src, JA_EN_TRANSLATE_TECHNICAL | JA_EN_TRANSLATE_GLIDERS)

print('done!')

Extracting tabular data from sources...done!


### Save extracted data to CSV

In [6]:
print('Saving extracted data to CSVs...', end='')

dframes = {
    'Character': character_stats_df,
    'Frame': frame_stats_df,
    'Tire': tire_stats_df,
    'Glider': glider_stats_df,
}

Path('../stats').mkdir(parents=True, exist_ok=True)
for (name, df) in dframes.items():
    df.to_csv(f'{ROOT_DIR}/stats/{name}.csv', index=False)
    
print('done!')

Saving extracted data to CSVs...

NameError: name 'ROOT_DIR' is not defined